In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import nltk
import os
import re
import string
import urllib.request
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation
from heapq import nlargest
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from collections import defaultdict
import string
from nltk.stem.wordnet import WordNetLemmatizer
glovef="glove.6B.300d.txt"
import random

In [3]:
df = pd.read_csv (r'transcripts.csv')
#df="The two most likely largest inventions of our generation are the Internet and the mobile phone. They've changed the world. However, largely to our surprise, they also turned out to be the perfect tools for the surveillance state. It turned out that the capability to collect data, information and connections about basically any of us and all of us is exactly what we've been hearing throughout of the summer through revelations and leaks about Western intelligence agencies, mostly U.S. intelligence agencies, watching over the rest of the world."
lemmatizer = WordNetLemmatizer()
stopword = set(stopwords.words('english'))
excluder = set(string.punctuation)
wordembd ={}
df.shape
df.head()


,transcript,url
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...
3,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...
4,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...


In [4]:
print( len(df.transcript), " speeches are available to summarize")

2467  speeches are available to summarize


In [5]:
print("Loading Glove Model")
f = open(glovef, encoding="utf8")
model = {}
for line in f:
    splitLine = line.split()
    word = splitLine[0] #it takes a word
    embedding = np.asarray(splitLine[1:], dtype='float32') #it shows the taken words word-word co-occurrence matrix values.
    model[word] = embedding # so we have our model of word embeddings
print("Done.",len(model)," vocabularies loaded!")


Loading Glove Model


FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.300d.txt'

In [ ]:
y=int(input())
print(y)

In [ ]:
b={}
l=len(df.transcript)
for i in range (y):
    R=random.randint(0,l)
    b[i]=R
print(b)

In [ ]:
for d in range (0,(y)):
    dt=df.transcript[b[d]] 
    sentences = sent_tokenize(dt)
    
   
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in sentences]
    dim=300
    svectors = []
    for i in stripped:
        if len(i) != 0:
            v = sum([model.get(w, np.zeros((dim,))) for w in i.split()])/(len(i.split()))
        else:
            v = np.zeros((dim,))
        svectors.append(v)
    
    similarity_matrix = np.zeros([len(stripped), len(stripped)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                similarity_matrix[i][j] = cosine_similarity(svectors[i].reshape(1,dim),svectors[j].reshape(1,dim))[0,0]
    
    print(similarity_matrix)
    print ('/////////////////////////////////  Original speech number',d+1 )
    print ('/////////////////////////////////  Original speech number',d+1 )
    print(df.transcript[b[d]])
    print ('/////////////////////////////////  summarized speech number',d+1 )
    print ('/////////////////////////////////  summarized speech number',d+1 )
    nx_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(nx_graph)
    top = sorted(((scores[i],i) for i,s in enumerate(sentences)), reverse=True) 
    arranged_sentences = sorted(top[0:2] )
    print("\n".join([sentences[x[1]] for x in arranged_sentences])) 
    print ('/////////////////////////////////  The end of summarization number',d+1 )
 